# Preprocesamiento de datos

## Objetivos particulares:

# ¿agregar aquí la regionalización en lugar de sección IX?

## Librerías

In [3]:
import pandas as pd
import numpy as np
import random
import copy
from sklearn.preprocessing import LabelEncoder

## 1. Cargar el conjunto de datos ENDIREH Sección IX: Atención Obstétrica

In [4]:
ruta_parcial = "../../conjunto_de_datos_endireh_2016_csv/conjunto_de_datos_tsdem_endireh_2016/conjunto_de_datos/conjunto_de_datos_tsdem_endireh_2016.csv"

endireh = pd.read_csv(ruta_parcial, na_values=['\r'])

In [5]:
endireh.iloc[random.choices(range(endireh.shape[0]), k=5)]
#endireh.iloc[[83867, 107779, 25309, 64234, 81171]]

,ID_VIV,ID_MUJ,UPM,PROG,VIV_SEL,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,HOGAR,...,REN_INF_AD,FN_DIA,FN_MES,FAC_VIV,FAC_MUJ,DOMINIO,ESTRATO,EST_DIS,UPM_DIS,COD_M15
389389,2802651.05,2802651.05.01.03\r,2802651,216,5,28,Tamaulipas\r,27,Nuevo Laredo\r,1,...,NaN,NaN,NaN,257,0,U\r,2,5190,15527,NaN
172226,1300935.03,1300935.03.01.02\r,1300935,71,3,13,Hidalgo\r,77,Tulancingo de Bravo\r,1,...,2.0,NaN,NaN,206,206,U\r,3,2290,7283,1.0
136103,1003069.07,1003069.07.01.01\r,1003069,400,7,10,Durango\r,12,Lerdo\r,1,...,1.0,NaN,NaN,161,161,C\r,2,1710,6153,1.0
213545,1562089.05,1562089.05.01.02\r,1562089,90,5,15,Estado de Mexico\r,118,Zinacantepec\r,1,...,2.0,NaN,NaN,1214,1214,R\r,1,2770,8816,1.0
37944,300881.20,0300881.20.01.03\r,300881,44,20,3,Baja California Sur\r,8,Los Cabos\r,1,...,NaN,NaN,NaN,62,0,C\r,2,400,1723,NaN


## 2. Modificar columnas a ser _One Hot Encoding_

Hacer que las columnas P1_4 y P1_8 en donde tienen el valor 2 sean 0s.

In [6]:
columnasOHE = ['P2_8', 'P2_9', 'P2_11', 'P2_12', 'P2_13']

for c in columnasOHE:
    endireh.loc[endireh[c]==2, c] = 0 #asignar los ceros de columnas con 2

### 3.2 Dividir por regiones de méxico.

Para no dividir entre 32 entidades federativas, dividiré el país en principales regiones de México.
Según la página de [Regiones de México](https://es.wikipedia.org/wiki/Regiones_de_M%C3%A9xico) de Wikipedia existen un total de 8 regiones.
   

|  Noroeste de México | Noreste de México  | Oeste de México  | Este de México  |
|:-------------------:|:------------------:|:----------------:|:---------------:|
| Baja California     | Coahuila           | Colima           | Hidalgo         |
| Baja California Sur | Nuevo León         | Jalisco          | Puebla          |
| Chihuahua           | Tamaulipas         | Michoacán        | Tlaxcala        |
| Durango             |                    | Nayarit          | Veracruz        |
| Sinaloa             |                    |                  |                 |
| Sonora              |                    |                  |                 |

|  Centronorte de México  | Centrosur de México  | Suroeste de México  | Sureste de México  |
|:-----:                  |:--------------------:|:-------------------:|:------------------:|
| Aguascalientes          | Ciudad de México     | Chiapas             | Campeche           |
| Guanajuato              | México               | Guerrero            | Quintana Roo       |
| Querétaro               | Morelos              | Oaxaca              | Tabasco            |
| San Luis Potosí         |                      |                     | Yucatán            |
| Zacatecas               |                      |                     |                    |

Haré un total de 8 archivos _csv_ en los que estarán en cada uno los datos de los estados correspondientes a la región del país.

Creo un código para rastrear el número de región a su significado.


|  Código  | Región         |
|:--------:|:--------------:|
| 0        | Noroeste       |
| 1        | Noreste        |
| 2        | Oeste          |
| 3        | Este           |
| 4        | Centronorte    |
| 5        | Centrosur      |
| 6        | Suroeste       |
| 7        | Sureste        |


Crear la nueva columna de región

In [7]:
def f(row):
    if   row['NOM_ENT'] in ['Baja California\r', 'Baja California Sur\r', 'Chihuahua\r', 'Durango\r', 'Sinaloa\r', 'Sonora\r']:
        val = 0
    elif row['NOM_ENT'] in ['Coahuila de Zaragoza\r', 'Nuevo Leon\r', 'Tamaulipas\r']:
        val = 1
    elif row['NOM_ENT'] in ['Colima\r', 'Jalisco\r', 'Michoacan de Ocampo\r', 'Nayarit\r']:
        val = 2
    elif row['NOM_ENT'] in ['Hidalgo\r', 'Puebla\r', 'Tlaxcala\r', 'Veracruz de Ignacio de la Llave\r']:
        val = 3
    elif row['NOM_ENT'] in ['Aguascalientes\r', 'Guanajuato\r', 'Queretaro\r', 'San Luis Potosi\r', 'Zacatecas\r']:
        val = 4
    elif row['NOM_ENT'] in ['Ciudad de Mexico\r', 'Estado de Mexico\r', 'Morelos\r']:
        val = 5
    elif row['NOM_ENT'] in ['Chiapas\r', 'Guerrero\r', 'Oaxaca\r']:
        val = 6
    elif row['NOM_ENT'] in ['Campeche\r', 'Quintana Roo\r', 'Tabasco\r', 'Yucatan\r']:
        val = 7
        
    return val

In [8]:
endireh_reg = copy.copy(endireh)
endireh_reg.insert(12, "REGION", endireh.apply(f, axis=1))
endireh_reg.head(10)

,ID_VIV,ID_MUJ,UPM,PROG,VIV_SEL,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,HOGAR,...,REN_INF_AD,FN_DIA,FN_MES,FAC_VIV,FAC_MUJ,DOMINIO,ESTRATO,EST_DIS,UPM_DIS,COD_M15
0,100006.04,0100006.04.01.02\r,100006,39,4,1,Aguascalientes\r,1,Aguascalientes\r,1,...,NaN,NaN,NaN,86,172,U\r,3,20,3,1.0
1,100025.03,0100025.03.02.03\r,100025,46,3,1,Aguascalientes\r,1,Aguascalientes\r,2,...,NaN,NaN,NaN,92,184,U\r,3,20,9,1.0
2,100031.05,0100031.05.01.03\r,100031,7,5,1,Aguascalientes\r,1,Aguascalientes\r,1,...,3.0,NaN,NaN,92,184,U\r,3,20,11,1.0
3,100035.03,0100035.03.02.02\r,100035,46,3,1,Aguascalientes\r,1,Aguascalientes\r,2,...,NaN,NaN,NaN,80,80,U\r,3,20,12,1.0
4,100041.01,0100041.01.01.03\r,100041,71,1,1,Aguascalientes\r,1,Aguascalientes\r,1,...,NaN,NaN,NaN,101,203,U\r,3,20,14,1.0
5,100126.01,0100126.01.01.02\r,100126,16,1,1,Aguascalientes\r,1,Aguascalientes\r,1,...,2.0,NaN,NaN,78,78,U\r,3,20,36,1.0
6,100133.02,0100133.02.01.01\r,100133,95,2,1,Aguascalientes\r,1,Aguascalientes\r,1,...,1.0,NaN,NaN,84,84,U\r,3,20,37,1.0
7,100141.03,0100141.03.01.03\r,100141,10,3,1,Aguascalientes\r,1,Aguascalientes\r,1,...,NaN,NaN,NaN,84,168,U\r,3,20,38,1.0
8,100143.01,0100143.01.01.02\r,100143,6,1,1,Aguascalientes\r,1,Aguascalientes\r,1,...,NaN,NaN,NaN,80,80,U\r,3,20,39,1.0
9,100143.03,0100143.03.01.02\r,100143,51,3,1,Aguascalientes\r,1,Aguascalientes\r,1,...,NaN,NaN,NaN,80,160,U\r,3,20,39,1.0


## 3. Borrar columnas

Borrar las columnas que ya están en otros conjuntos de datos, para no ser redundantes.

In [11]:
labels = ['CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'HOGAR', 'COD_RES', 'COD_RES_E', 'FN_DIA', 'FN_MES',
          'DOMINIO', 'UPM', 'VIV_SEL', 'PROG', 'N_REN', 'REN_MUJ_EL', 'REN_INF_AD', 'FAC_VIV', 'FAC_MUJ',
          'ESTRATO', 'EST_DIS', 'UPM_DIS', 'CODIGO', 'COD_M15', 'NOMBRE', 'P2_16']

endireh_reg.drop(labels=labels, axis=1, inplace=True)

## 4. Guardar el datasets

In [18]:
#guardo los avances
endireh_reg.to_csv("datasets/sec_ii.csv")